In [16]:
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, ElementClickInterceptedException, ElementNotInteractableException
# from bs4 import BeautifulSoup
import pandas as pd
import time

# Enter Data to To search form

#specify driver path
browser = webdriver.Chrome(executable_path= r'C:\ChromeDriver\chromedriver.exe')
browser.get('https://www.indeed.fr/Paris-(75)-Emplois')
# browser.maximize_window()
advanced_search = browser.find_element_by_xpath("//a[contains(text(),'Recherche avancée')]")
advanced_search.click()


#search data science 
search_job = browser.find_element_by_xpath('//input[@id="as_and"]')
search_job.clear()
search_job.send_keys(['Data Scientist'])

# search Specific skills
specific_skills = browser.find_element_by_xpath('//input[@id="as_phr"]')
specific_skills.clear()
# specific_skills.send_keys(['NLP'])


#set display limit of 30 results per page
display_limit = browser.find_element_by_xpath('//select[@id="limit"]//option[@value="30"]')
display_limit.click()

#sort by date
sort_option = browser.find_element_by_xpath('//select[@id="sort"]//option[@value="date"]')
sort_option.click()

# pick alternance
# sort_option = browser.find_element_by_xpath('//select[@id="jt"]//option[@value="custom_1"]')
# sort_option.click()

#  remove cookie banner because it covers the next research button
# try:
#     cookie_banner = browser.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]')
#     cookie_banner.click()
# except NoSuchElementException:
#     pass
    
search_button = browser.find_element_by_xpath('//*[@id="fj"]')
search_button.click()


# handle pop up 
# wait about 2 seconds

time.sleep(2)
try:
    close_popup = browser.find_element_by_id("popover-x")
    close_popup.click()
except (NoSuchElementException,NoSuchWindowException) as e:
    pass
    


In [17]:
df = pd.DataFrame(columns=["ID","Title","Location",
                           "Company","Salary","Description","Links"])
"""
    df_indeed = {'ID': [], 'Title': [], 'Location': [], 'Company': [], 'Links': [], 'Salary': [], 'Descriptions': [],
               'Python': [], 'R': [], 'SQL': [], 'NoSQL': [], 'GIT': [], 'Spark': [], 'Flask': [], 'Streamlit': [], 'Docker': [], 'Kubernetes': [],
               'React': [], 'VueJS': [], 'AngularJS': [],
               'Machine Learning': [], 'Deep Learning': [], 'NLP': [],  'Scala': [], 'PySpark': [],
               'PowerBI': [], 'SQLServer': [], 'Dataiku': [], 'Keras': [], 'TensorFlow': [], 'NLU': [],
               'PyTorch': [], 'ScikitLearn': [], 'Scikit-Learn': [], 'SAS': [],
               'Java': [], 'Scikit learn': [], 'Hadoop': [],  'Hive': [], 'ML DL': [], 'Azure': [], 'AWS': []
               }
 """

while True :
    # Cookies and Popup Handling*
    try:
        time.sleep(1)
        if browser.find_element_by_id("onetrust-accept-btn-handler"):
            browser.find_element_by_id("onetrust-accept-btn-handler").click()
    except:
        pass
    try:
        time.sleep(1)
        if browser.find_element_by_css_selector("div.popover-foreground div.popover-x"):
            browser.find_element_by_css_selector("div.popover-foreground div.popover-x").click()
    except:
        pass
    
    job_card = browser.find_elements_by_css_selector('div.jobsearch-SerpJobCard')

    for job in job_card:

        # get the id of each job application to filter out the duplicated job offers:
        job_id=job.get_attribute('id')

        # get the job title 
        try:
            job_title =job.find_element_by_css_selector('h2.title a').get_attribute('title')

        except:
            job_title = 'None'
        # get the job descriptions
        try:
            job.click()
            time.sleep(1)
            job_desc=browser.find_element_by_id('vjs-desc').text.strip()
            # to close the description window to percept the next button and click o it  
            time.sleep(0.25)
#                 browser.find_element_by_id('vjs-x').click()

        except:
            job_desc='None'

        # get the job application location 
        try:
            job_location =job.find_element_by_css_selector('div.recJobLoc').get_attribute('data-rc-loc')

        except:
            job_location='None'

        # get the job company name 
        try:
            job_company=job.find_element_by_css_selector('div span.company').text

        except:
            job_company='None'

        # get the salary proposed by the company 
        try:
            job_salary=job.find_element_by_css_selector('span.salaryText').text
        except:
            job_salary='None'
            
        # get job links 
        try:
            job_link=job.find_element_by_css_selector("h2.title > a").get_attribute("href")
            
        except:
            job_link='None'
            


        df = df.append({"ID":job_id,'Title':job_title,'Location':job_location,"Company":job_company,"Salary":job_salary,
                        "Description":job_desc,'Links':job_link},ignore_index=True)
        
   
        
    # scroll down to see the next button 
    browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")

    
    # to get the next button because it was interceptable because a tag was hiding it 
    try:
        next_page = browser.find_element_by_xpath("//a[@aria-label='Suivant'] | a[contains(text(),'Suivant')] ").get_attribute(name="href")
        browser.get(next_page)
    except NoSuchElementException:
        break
        print("End")

    except TimeoutException:
        print("Toutes les pages ont été visité")
        break
    

# store data into csv format
df.to_csv('data2.csv')
df.head()

,ID,Title,Location,Company,Salary,Description,Links
0,p_17ea7986b1f5b00d,DATA ANALYST,Villepinte (93),INFORMATIS-TS,35 000 € - 55 000 € par an,Dans le cadre d’un important projet avec une e...,https://www.indeed.fr/company/INFORMATIS--TS/j...
1,p_05c88458c5d9c869,Senior Data Analyst,Courbevoie (92),IQVIA,None,"Data Analyst, Real World & Analytics Solutions...",https://www.indeed.fr/rc/clk?jk=05c88458c5d9c8...
2,p_b117b0b7559e2118,Stages Data Science,Ivry-sur-Seine (94),AfterData,None,Stage - Ivry-sur-Seine (Métro)\nPassionné par ...,https://www.indeed.fr/rc/clk?jk=b117b0b7559e21...
3,p_1cd2c87335ba070c,Data scientist-(H/F),Courbevoie (92),Société Générale,None,Vos missions au quotidien\nNous vous proposons...,https://www.indeed.fr/rc/clk?jk=1cd2c87335ba07...
4,p_49313c8d820629d9,Data scientist F/H,Courbevoie (92),autobiz,None,"Créée en 2004, autobiz est une start-up innova...",https://www.indeed.fr/rc/clk?jk=49313c8d820629...


In [ ]:
import pandas as pd 
df=pd.read_csv('dataIndeed.csv',index_col='Unnamed: 0')

In [30]:
df.columns


Index(['index', 'ID', 'Title', 'Location', 'Company', 'Salary', 'Description',
       'Links'],
      dtype='object')

In [3]:
# https://stackoverflow.com/questions/32588797/pandas-how-to-apply-a-function-to-different-columns
def lower_func(x):
    return x.astype(str).str.lower()
df[['Title', 'Location', 'Company', 'Salary', 'Description']]=df[['Title', 'Location', 'Company', 'Salary', 'Description']].apply(lower_func,axis=1)
df.head()



,index,ID,Title,Location,Company,Salary,Description,Links
0,0,p_e1902be83a5c3cee,senior business data analyst,paris (75),civils de la défense,47 700 € - 50 900 € par an,none,https://www.indeed.fr/rc/clk?jk=e1902be83a5c3c...
1,1,p_6c20aebcee85ac64,data analyst - h/f,paris 19e (75),golden bees,38 000 € par an,r / microsoft-excel / sql\nanalyste data marke...,https://www.indeed.fr/rc/clk?jk=6c20aebcee85ac...
2,2,p_f402d66f9081622f,data analyst - boulogne f/h,boulogne-billancourt (92),cegedim,none,l'équipe r&d du groupe cegedim évalue et met e...,https://www.indeed.fr/rc/clk?jk=f402d66f908162...
3,3,p_0c3c13bdb7039f10,data scientist - f/h,la défense (92),ingeniance,none,"contexte:\nvous assurez la production, la conc...",https://www.indeed.fr/rc/clk?jk=0c3c13bdb7039f...
4,4,p_8acc248a08d4f035,data analyst (h/f) - boulogne (france),boulogne-billancourt (92),cegedim,none,l’équipe r&d du groupe cegedim évalue et met e...,https://www.indeed.fr/rc/clk?jk=8acc248a08d4f0...


# Nettoyage de données 

In [4]:
 skills=['python', 'r', 'sql', 'Nosql', 'git', 'spark', 'flask', 'streamlit', 'docker', 'kubernetes',
               'react', 'vuejs', 'angularjs',     'machine learning', 'deep learning', 'nlp',  'scala', 'pyspark',
               'powerbi', 'sqlserver', 'dataiku', 'keras', 'tensorflow', 'nlu',
               'pytorch', 'scikitlearn', 'scikit-learn', 'sas',
               'java', 'scikit learn', 'hadoop',  'hive', 'ml dl', 'azure', 'aws']
    
df['Skills'] = [list() for x in range(len(df.index))]
for index, values in df.Description.iteritems():
#     print(values.split(' '))
#     print('--------------------------------------------------------------------------')
    for word in values.split(' '):
        if word in skills:
            if word not in df['Skills'][index]:
#                 print(word, index)
                df['Skills'][index].append(word)
            

In [34]:
for index,i in df['Skills'].iteritems():
    if 'nlp' in i :
        print(i,index)


['python', 'hadoop', 'sql', 'nlp'] 5
['nlp'] 65
['nlp', 'java'] 66
['nlp'] 68
['python', 'nlp'] 77
['nlp', 'python', 'tensorflow', 'sql'] 154
['nlp', 'python'] 229
['nlp', 'python'] 273
['python', 'nlp'] 275
['hadoop', 'nlp', 'sql'] 341
['nlp'] 342
['nlp', 'sql', 'r'] 343
['hadoop', 'nlp'] 344
['nlp', 'pyspark'] 345
['nlp'] 347
['nlp'] 349
['nlp'] 350
['nlp'] 351
['nlp', 'python'] 392


In [85]:
# df['Skills'] = [skill for skill in skills if df.Description.strip().isin(skills).astype(str)]
# df['Skills']

## NLP Profiler

In [1]:

# %%bash

# pip install git+https://github.com/neomatrix369/nlp_profiler.git@master
# echo "Once successfully installed, please restart your Jupyter kernels for the changes to take effect"

In [1]:
# from nlp_profiler.core import apply_text_profiling


# Connect to Mongo db and Push Data into 

In [3]:
### convertion dataframe en dictionnaire 
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_dict.html
df.reset_index(inplace=True)
data_dict = df.to_dict("records")


import pymongo 
from pymongo import MongoClient



# # En amont j'ai créer la database jobapplications dans un cmd

 

# Start MongoDB
MONGO_HOST= 'mongodb://localhost:27017/'

 

#MongoClient is used to communicate with MongoDB
client = MongoClient(MONGO_HOST)



#We get a reference to the jobapplications database

 

db=client['job_applications']

 

# create a gene collection and insert jobapplications data 
db.Indeed.insert_many(data_dict)


# Requete SQL

https://api.mongodb.com/python/current/tutorial.html



In [79]:
# printer les 
import pprint
from pprint import PrettyPrinter
pp = PrettyPrinter()
for document in db.Indeed.find():
    pp.pprint(document)
    break

{'Company': 'Civils de la Défense',
 'Description': 'None',
 'ID': 'p_e1902be83a5c3cee',
 'Links': 'https://www.indeed.fr/rc/clk?jk=e1902be83a5c3cee&fccid=da943da1ba1f181d&vjs=3',
 'Location': 'Paris (75)',
 'Salary': '47 700 € - 50 900 € par an',
 'Title': 'SENIOR BUSINESS DATA ANALYST',
 '_id': ObjectId('5f5a138147aa23246ebab4c0'),
 'index': 0}


In [ ]:
from pprint import PrettyPrinter
pp = PrettyPrinter()
pp.pprint(new_file)

In [77]:
db.list_collection_names()

['Monster', 'Indeed']

In [81]:
# pour afficher une ligne (qu'on appel document)
pprint.pprint(db.Indeed.find_one())

{'Company': 'Civils de la Défense',
 'Description': 'None',
 'ID': 'p_e1902be83a5c3cee',
 'Links': 'https://www.indeed.fr/rc/clk?jk=e1902be83a5c3cee&fccid=da943da1ba1f181d&vjs=3',
 'Location': 'Paris (75)',
 'Salary': '47 700 € - 50 900 € par an',
 'Title': 'SENIOR BUSINESS DATA ANALYST',
 '_id': ObjectId('5f5a138147aa23246ebab4c0'),
 'index': 0}


In [84]:
for job in db.Indeed.find({"Description": "data"}):
    pprint.pprint(job)

In [126]:
# result=db.Indeed.aggregate([{ '$match': { 'Company': "data" } },{'$limit': 5}])
# for i, o in enumerate(result):
#     print(i, o)

In [ ]:
# { $group: { _id: "$cust_id", total: { $sum: "$amount" } } }

In [95]:
db=client['job_applications']
mycol = db["Indeed"]

In [118]:
   myquery = {
    "Company" : { "$in": ["phenix", "edf"] },
    "Location" : { "$regex": "paris" }
}
    
mydoc = mycol.find(myquery)
for x in mydoc:
    pprint.pprint(x)

{'Company': 'phenix',
 'Description': 'data analyst\n'
                '\n'
                'l’entreprise : phenix est la startup leader de la lutte '
                'contre le gaspillage en europe, avec plus de 50 millions de '
                'repas sauvés depuis son lancement grâce à son réseau de '
                'professionnel·les et citoyen·nes engagé·es.\n'
                'localisation : paris 17e - télétravail possible 3 '
                'jours/semaine après période d’essai\n'
                'disponibilité : octobre 2020\n'
                'type de contrat : cdi\n'
                'rémunération : compétitive et en fonction du profil + tickets '
                'restaurant + 100% abonnement transport en commun\n'
                'site web : www.wearephenix.com\n'
                '\n'
                'rejoignez phenix pour participer à la réduction du gaspillage '
                'et à la transition vers une économie circulaire pour ne plus '
                'nourrir les pou

In [121]:
# https://www.it-swarm.dev/fr/regex/comment-utiliser-loperateur-not-dans-mongodb/1042778394/
# matching pattern without regex :   https://www.guru99.com/regular-expressions-mongodb.html
# https://kb.objectrocket.com/mongo-db/how-to-query-mongodb-documents-with-regex-in-python-362
myquery = {
    "Location" : { "$regex": "paris" },
    "Description" : { "$regex": "nlp" }
}
    
mydoc2 = mycol.find(myquery)
for x in mydoc2:
    pprint.pprint(x)

{'Company': 'scient',
 'Description': 'nous cherchons notre data scientist\n'
                'la mission : comprendre le métier et le traduire en '
                'spécifications techniques. en étroite collaboration avec une '
                'petite équipe vous serez en charge :\n'
                'elaboration de projet et programmation en data-science\n'
                'participer au développement des solutions\n'
                'mise en place des technologies d’ia et de nlp\n'
                '\n'
                'profil recherché les prérequis : vous justifiez '
                'impérativement d’une première expérience en python/js, vous '
                'êtes diplômé d’un master en informatique ou d’un diplôme '
                'd’ingénieur (ingénieur informaticien) et vous maîtrisez au '
                'moins deux des compétences suivantes :\n'
                'algo : python (tornado, flask), javascript (nodejs)\n'
                'data : mongodb, arangodb, postgresql, elas

In [124]:
db.Indeed.find().sort({'ID':-1}).limit(2).forEach("printjson")

TypeError: if no direction is specified, key_or_list must be an instance of list